In [44]:
import pandas as pd
import csv
from dataset import read_sentiment_data

# Data Exploration

In [45]:
df_train = read_sentiment_data("./data/sentiment/train.txt")

In [46]:
df_train = pd.DataFrame(data=df_train)
df_train = df_train.convert_dtypes()

In [47]:
print(df_train) 

        id  label                                              value
0        0      1                          Cần tư vấn mà add  k rep

1        1      1         Hotline khó gọi quá gọi mãi ko thưa máy à

2        2      1  Mình thấy câu dịch vụ tốt nhất cho kh khó lắm....
3        3      1  Em chọn chuyển tiền trong nước. Chuyển đến số ...
4        4      1       Mình xài cái thể VISA của BIDV hạn mức 100tr
...    ...    ...                                                ...
1972  1972      2                                      Dạ em cảm ơn

1973  1973      1  Có kinh nghiệm nhưng phải bằng đại học chính q...
1974  1974      2                     Vietcombank tks add trước nha

1975  1975      2                            Vietcombank ok tks add

1976  1976      1                  Gọi k được mà tốn tiền như gì ấy


[1977 rows x 3 columns]


In [48]:
positives = df_train['label'][df_train['label'] == 2].count()
negatives = df_train['label'][df_train['label'] == 1].count()
neutrals = df_train['label'][df_train['label'] == 0].count()

print(f"positive label count: {positives}")
print(f"negative label count: {negatives}")
print(f"neutral label count: {neutrals}")

print(f"total: {positives + negatives + neutrals} == {df_train['label'].count()}")

positive label count: 1211
negative label count: 743
neutral label count: 23
total: 1977 == 1977


# Model 

In [49]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained(
    "5CD-AI/Vietnamese-Sentiment-visobert")

## Tokenizer

In [50]:
tokenizer = AutoTokenizer.from_pretrained(
    "5CD-AI/Vietnamese-Sentiment-visobert")

In [51]:
vocab = tokenizer.vocab
reversed_vocab = {v: k for k, v in vocab.items()} # reverse it so that we can retrieve the text from the token 
tokenizer

XLMRobertaTokenizerFast(name_or_path='5CD-AI/Vietnamese-Sentiment-visobert', vocab_size=15002, model_max_length=256, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	15001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [52]:
df_train.loc[0].value

'Cần tư vấn mà add  k rep\n'

In [53]:
t = tokenizer.encode(text=df_train.loc[0].value)

In [54]:
for id in t: 
    print(f"Token: {id} -> {reversed_vocab[id]}") # ignores new line character, a property of sentencepiece 

Token: 0 -> <s>
Token: 2615 -> ▁Cần
Token: 749 -> ▁tư
Token: 970 -> ▁vấn
Token: 50 -> ▁mà
Token: 2786 -> ▁add
Token: 17 -> ▁k
Token: 2321 -> ▁rep
Token: 2 -> </s>


In [55]:
s = tokenizer.decode(t, skip_special_tokens=True) 
print(s)

Cần tư vấn mà add k rep
